<a href="https://colab.research.google.com/github/arina19-2000/unimi/blob/main/tesi/tesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/colab


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/colab


## Data preparation

In [ ]:
dfa = pd.read_csv('results adults2.csv')

In [ ]:
column_types = dfa.dtypes

type_counts = column_types.value_counts()
print("Column types and counts:")
print(type_counts)

Column types and counts:
object     53
float64    50
int64       3
Name: count, dtype: int64


In [ ]:
dfa = dfa.dropna(axis=1, how='all')
dfa.shape

(786, 109)

In [ ]:
dfa = dfa.dropna(thresh=int(0.05 * dfa.shape[1]), axis=0)
dfa.shape

(495, 109)

In [ ]:
dfa = dfa.dropna(thresh=int(0.05 * dfa.shape[1]), axis=1)
dfa.shape

(495, 105)

In [ ]:
dfa['Qual è il titolo di studio massimo conseguito?'] = dfa['Qual è il titolo di studio massimo conseguito?'].str.extract('(\d)')
dfa['Quale scuola secondaria di secondo grado (o superiore) ha frequentato?'] = dfa['Quale scuola secondaria di secondo grado (o superiore) ha frequentato?'].str.extract('(\d)')
dfa['Perché ha lasciato?']=dfa['Perché ha lasciato?'].str.extract('(\d)')
dfa.head()

,ID risposta,Ha uno o più figli adottati nati prima del 2002 (escluso)?,Numero di figli adottati nati prima del 2002 (escluso) [N°][],Genere,Anno di nascita [Anno di nascita][],Adozione,Anno di arrivo in famiglia [Anno][],Stato di provenienza,Era scolarizzato nel paese di origine (Italia inclusa)?,Età di inserimento a scuola [Età][],...,Con quale valutazione? [Voto][.../60].3,Ha conseguito una laurea triennale o magistrale?.3,Voto della laurea? [Voto][.../110],Professione del PADRE quando il figlio ha fatto la scelta della scuola secondaria di II° grado o superiore:,Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [No],"Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [Sì, alle primarie]","Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [Sì, alla secondaria di primo grado]","Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [Sì, alla secondaria di secondo grado]",Il PADRE ha dovuto ripetere degli anni alla scuola secondaria di secondo grado (o superiori)?,Il PADRE ha avuto difficoltà di rendimento/apprendimento che lo hanno spinto a cambiare tipologia di indirizzo di scuola secondaria di secondo grado (o superiore)?
0,1,Sì,1.0,Femmina,1999.0,Nazionale,2004.0,NaN,No,5.0,...,NaN,Laurea magistrale o vecchio ordinamento,110.0,programmista,No,No,No,Sì,"Sì, 1 anno",Sì
2,3,Sì,2.0,Femmina,1990.0,Nazionale,1991.0,Italia,No,6.0,...,NaN,NaN,NaN,Operaio,Sì,No,No,No,NaN,NaN
3,4,Sì,1.0,Femmina,1996.0,Internazionale,2003.0,Ucraina,No,6.0,...,NaN,Laurea magistrale o vecchio ordinamento,110.0,Impiegato,Sì,No,No,No,No,No
4,5,Sì,1.0,Femmina,1996.0,Nazionale,2001.0,Italia,Sì,4.0,...,NaN,NaN,110.0,Assistente sociale,No,No,No,Sì,"Sì, 1 anno",No
5,6,Sì,1.0,Femmina,1999.0,Internazionale,2014.0,Ucraina,No,6.0,...,60.0,NaN,NaN,Impiegato statale,Sì,No,No,No,No,No


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('italian'))

def remove_italian_stopwords(col):
    col = col.lower()
    col = re.sub(r'[^\w\s]', '', col)
    # col = re.sub(r'^G\d+\[.*?\]\.\s*', '', col)
    col = re.sub(r'\s+', '_', col)
    words = [word for word in col.split('_') if word not in stop_words]
    return '_'.join(words)

dfa.columns = [remove_italian_stopwords(col) for col in dfa.columns]

dfa.columns


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Index(['id_risposta', 'figli_adottati_nati_prima_2002_escluso',
       'numero_figli_adottati_nati_prima_2002_escluso_n', 'genere',
       'anno_nascita_anno_nascita', 'adozione', 'anno_arrivo_famiglia_anno',
       'stato_provenienza', 'scolarizzato_paese_origine_italia_inclusa',
       'età_inserimento_scuola_età',
       ...
       'valutazione_voto603', 'conseguito_laurea_triennale_magistrale3',
       'voto_laurea_voto110',
       'professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_no',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado',
       'padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori',
       'padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipolo

In [ ]:
dfa = dfa.loc[:, dfa.apply(pd.Series.nunique) > 1]
dfa.shape

(495, 103)

In [ ]:
dfa.head()

,id_risposta,numero_figli_adottati_nati_prima_2002_escluso_n,genere,anno_nascita_anno_nascita,adozione,anno_arrivo_famiglia_anno,stato_provenienza,scolarizzato_paese_origine_italia_inclusa,età_inserimento_scuola_età,prima_classe_frequenza_scuola,...,valutazione_voto603,conseguito_laurea_triennale_magistrale3,voto_laurea_voto110,professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore,padre_mai_stato_organi_collegiali_scuole_propri_figli_no,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore
0,1,1.0,Femmina,1999.0,Nazionale,2004.0,NaN,No,5.0,Scuola materna,...,NaN,Laurea magistrale o vecchio ordinamento,110.0,programmista,No,No,No,Sì,"Sì, 1 anno",Sì
2,3,2.0,Femmina,1990.0,Nazionale,1991.0,Italia,No,6.0,Prima elementare,...,NaN,NaN,NaN,Operaio,Sì,No,No,No,NaN,NaN
3,4,1.0,Femmina,1996.0,Internazionale,2003.0,Ucraina,No,6.0,Prima elementare,...,NaN,Laurea magistrale o vecchio ordinamento,110.0,Impiegato,Sì,No,No,No,No,No
4,5,1.0,Femmina,1996.0,Nazionale,2001.0,Italia,Sì,4.0,Scuola materna,...,NaN,NaN,110.0,Assistente sociale,No,No,No,Sì,"Sì, 1 anno",No
5,6,1.0,Femmina,1999.0,Internazionale,2014.0,Ucraina,No,6.0,Scuola materna,...,60.0,NaN,NaN,Impiegato statale,Sì,No,No,No,No,No


In [ ]:
dfa.padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie.nunique()

2

## Rename column content

In [ ]:
dfa['genere_fem'] = dfa['genere'].apply(lambda x: 1 if x == 'Femmina' else 0)
dfa['adozione_int'] = dfa['adozione'].apply(lambda x: 1 if x == 'Internazionale' else 0)

In [ ]:
dfa['adopt_age'] = dfa['anno_arrivo_famiglia_anno'] - dfa['anno_nascita_anno_nascita']


In [ ]:
dfa.head()

,id_risposta,numero_figli_adottati_nati_prima_2002_escluso_n,genere,anno_nascita_anno_nascita,adozione,anno_arrivo_famiglia_anno,stato_provenienza,scolarizzato_paese_origine_italia_inclusa,età_inserimento_scuola_età,prima_classe_frequenza_scuola,...,professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore,padre_mai_stato_organi_collegiali_scuole_propri_figli_no,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore,genere_fem,adozione_int,adopt_age
0,1,1.0,Femmina,1999.0,Nazionale,2004.0,NaN,No,5.0,Scuola materna,...,programmista,No,No,No,Sì,"Sì, 1 anno",Sì,1,0,5.0
2,3,2.0,Femmina,1990.0,Nazionale,1991.0,Italia,No,6.0,Prima elementare,...,Operaio,Sì,No,No,No,NaN,NaN,1,0,1.0
3,4,1.0,Femmina,1996.0,Internazionale,2003.0,Ucraina,No,6.0,Prima elementare,...,Impiegato,Sì,No,No,No,No,No,1,1,7.0
4,5,1.0,Femmina,1996.0,Nazionale,2001.0,Italia,Sì,4.0,Scuola materna,...,Assistente sociale,No,No,No,Sì,"Sì, 1 anno",No,1,0,5.0
5,6,1.0,Femmina,1999.0,Internazionale,2014.0,Ucraina,No,6.0,Scuola materna,...,Impiegato statale,Sì,No,No,No,No,No,1,1,15.0


In [ ]:
for col in dfa.columns:
    if isinstance(dfa[col], pd.Series):
        unique_values = dfa[col].dropna().unique()
        if set(unique_values).issubset({'Sì', 'No'}):
            dfa[col] = dfa[col].replace({'Sì': 1, 'No': 0})



dfa.head()

,id_risposta,numero_figli_adottati_nati_prima_2002_escluso_n,genere,anno_nascita_anno_nascita,adozione,anno_arrivo_famiglia_anno,stato_provenienza,scolarizzato_paese_origine_italia_inclusa,età_inserimento_scuola_età,prima_classe_frequenza_scuola,...,professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore,padre_mai_stato_organi_collegiali_scuole_propri_figli_no,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore,genere_fem,adozione_int,adopt_age
0,1,1.0,Femmina,1999.0,Nazionale,2004.0,NaN,0.0,5.0,Scuola materna,...,programmista,0.0,0.0,0.0,1.0,"Sì, 1 anno",1.0,1,0,5.0
2,3,2.0,Femmina,1990.0,Nazionale,1991.0,Italia,0.0,6.0,Prima elementare,...,Operaio,1.0,0.0,0.0,0.0,NaN,NaN,1,0,1.0
3,4,1.0,Femmina,1996.0,Internazionale,2003.0,Ucraina,0.0,6.0,Prima elementare,...,Impiegato,1.0,0.0,0.0,0.0,No,0.0,1,1,7.0
4,5,1.0,Femmina,1996.0,Nazionale,2001.0,Italia,1.0,4.0,Scuola materna,...,Assistente sociale,0.0,0.0,0.0,1.0,"Sì, 1 anno",0.0,1,0,5.0
5,6,1.0,Femmina,1999.0,Internazionale,2014.0,Ucraina,0.0,6.0,Scuola materna,...,Impiegato statale,1.0,0.0,0.0,0.0,No,0.0,1,1,15.0


In [ ]:
dfa = dfa.drop(columns=['genere','adozione','scuola_secondaria_secondo_grado_superiore_frequentato_altro', ])

In [ ]:
object_columns = dfa.columns[dfa.dtypes == 'object']
print(object_columns)

Index(['genere', 'adozione', 'stato_provenienza',
       'prima_classe_frequenza_scuola', 'prima_classe_frequenza_scuola_altro',
       'qual_titolo_studio_massimo_conseguito',
       'scuola_secondaria_secondo_grado_superiore_frequentato',
       'scuola_secondaria_secondo_grado_superiore_frequentato_altro',
       'dovuto_ripetere_anni_scuola_secondaria_superiore',
       'conseguito_laurea_triennale_magistrale',
       'ambito_titolo_universitario_es_economia_giurispudenza_medicina_etc',
       'dovuto_ripetere_anni_scuola_secondaria_superiore1', 'lavoro_fa',
       'attualmente_frequentando', 'fatto_esperienze_allestero',
       'descrizione_nucleo_familiare_figlio',
       'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_primaria',
       'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rap

In [ ]:
# categorical_cols = dfa.select_dtypes(include=['object']).columns

# for col in categorical_cols:
#     if col in dfa.columns:
#         dfa[col] = dfa[col].fillna('No response')
#     else:
#         print(f"Column {col} does not exist in the DataFrame.")

##Children survey results

In [ ]:
dfc = pd.read_excel('results kids.xlsx')

In [ ]:
dfc.shape

(235, 93)

In [ ]:
dfc = dfc.dropna(axis=1, how='all')
dfc.shape

(235, 91)

In [ ]:
dfc.columns = [remove_italian_stopwords(col) for col in dfc.columns]

dfc.columns

Index(['id_id_risposta', 'submitdate_data_invio', 'lastpage_ultima_pagina',
       'startlanguage_lingua_iniziale', 'seed_seme', 'startdate_data_inizio',
       'datestamp_data_dellultima_azione', 'refurl_url_riferimento',
       'privacysq001_base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_leg

In [ ]:
column_types = dfc.dtypes
type_counts = column_types.value_counts()

print("Column types and counts:")
print(type_counts)

Column types and counts:
object            72
float64           14
datetime64[ns]     3
int64              2
Name: count, dtype: int64


In [ ]:
categorical_cols = dfc.select_dtypes(include=['object']).columns

for col in categorical_cols:
    if col in dfc.columns:
        dfc[col] = dfc[col].fillna('No response')
    else:
        print(f"Column {col} does not exist in the DataFrame.")

In [ ]:
dfc.head()

,id_id_risposta,submitdate_data_invio,lastpage_ultima_pagina,startlanguage_lingua_iniziale,seed_seme,startdate_data_inizio,datestamp_data_dellultima_azione,refurl_url_riferimento,privacysq001_base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_legale_via_liberiana_n17_roma_infocoordinamentocareorg_persona_legale_rappresentante_preghiamo_quindi_manifestare_consenso_scritto_trattamento_parte_dati_classificati_sensibili_cliccando_caselle_qui_sotto_doppia_autorizzazione_requisito_necessario_proseguimento_questionario_letto_obbligatoria,privacysq002_base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_legale_via_liberiana_n17_roma_infocoordinamentocareorg_persona_legale_rappresentante_preghiamo_quindi_manifestare_consenso_scritto_trattamento_parte_dati_classificati_sensibili_cliccando_caselle_qui_sotto_doppia_autorizzazione_requisito_necessario_proseguimento_questionario_manifesto_consenso_obbligatoria,...,g8q00003a_conseguito_laurea_triennale_magistrale,g8q00004sq001_sq001_voto_laurea_voto110,g8q00005_professione_padre_quando_fatto_scelta_scuola_secondaria_ii_grado_superiore,g8q00006sq001_padre_mai_stato_organi_collegiali_scuole_propri_figli_no,g8q00006sq002_padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,g8q00006sq005_padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,g8q00006sq004_padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,g8q00007_padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,g8q00008_padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore,g8q00009_tipologia_cambio
0,1,NaT,0.0,it,433164667,2021-07-03 08:57:45,2021-07-03 08:57:51,No response,No,No,...,No response,NaN,No response,No response,No response,No response,No response,No response,No response,No response
1,2,2021-07-06 16:52:01,5.0,it,696014843,2021-07-06 16:48:47,2021-07-06 16:52:01,http://m.facebook.com/,Sì,Sì,...,No response,NaN,Impiegato,Sì,No,No,No,No response,No response,No response
2,3,2021-07-06 16:55:44,5.0,it,531131002,2021-07-06 16:49:03,2021-07-06 16:55:44,No response,Sì,Sì,...,No response,110.0,Docente universitario,Sì,No,No,No,No,No,No response
3,4,NaT,0.0,it,1471963680,2021-07-06 16:49:26,2021-07-06 16:49:39,http://m.facebook.com/,No,No,...,No response,NaN,No response,No response,No response,No response,No response,No response,No response,No response
4,5,NaT,2.0,it,1652562375,2021-07-06 16:54:19,2021-07-06 16:54:47,https://m.facebook.com/,Sì,Sì,...,No response,NaN,No response,No response,No response,No response,No 